In [1]:
%pip install tensorflow==2.12.0 opencv-python mediapipe pygame numpy

  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
                                              0.0/50.5 MB ? eta -:--:--
                                              0.1/50.5 MB 2.8 MB/s eta 0:00:18
                                              0.3/50.5 MB 2.8 MB/s eta 0:00:18
                                              0.4/50.5 MB 2.8 MB/s eta 0:00:19
                                              0.6/50.5 MB 3.0 MB/s eta 0:00:17
                                              0.7/50.5 MB 3.0 MB/s eta 0:00:17
                                              0.8/50.5 MB 2.9 MB/s eta 0:00:18
                                              1.0/50.5 MB 3.0 MB/s eta 0:00:17
                                              1.1/50.5 MB 2.9 MB/s eta 0:00:18
                                              1.2/50.5 MB 2.9 MB/s eta 0:00:17
     -                                        1.3/50.5 MB 2.8 MB/s eta 0:00:18
     -                                        1.4/50.5 MB 2.7 MB/s

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [17]:
import sys
import cv2
import numpy as np
import os
import time
import mediapipe as mp
import pygame

In [18]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [19]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [20]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [21]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [22]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect. Note that actions[0] == 'No-Pose', actions[1] == 'L-Pose' and so on...
actions = np.array(['No-Pose', 'L-Pose', 'X-Pose', 'Namaste-Pose'])

# Videos are going to be 30 frames in length
sequence_length = 30

In [23]:
label_map = {label:num for num, label in enumerate(actions)}

In [24]:
label_map

{'No-Pose': 0, 'L-Pose': 1, 'X-Pose': 2, 'Namaste-Pose': 3}

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [26]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.load_weights('action.h5')

# Prediction Cell

In [30]:
sequence = []
threshold = 0.9999
threshold1 = 0.9
threshold2 = 0.8

pygame.init()
pygame.mixer.init()
pose_achieved=pygame.mixer.Sound("pach.wav")
instructions=pygame.mixer.Sound("instruct.wav")
BeepOnePerSec=pygame.mixer.Sound("BeepOnePerSec.wav")
BeepTwoPerSec=pygame.mixer.Sound("BeepTwoPerSec.wav")

# Once we call instructions.play() - the audio will start playing. And the code will move on to the
# next line.
instructions.play()

cap = cv2.VideoCapture(0)
# Set mediapipe model, note  that we set a high bar for detection confidence. this is just
# saying that, we want the model to be absolutely sure that it detected a pose (with landmarks)
# before it claims victory. we would rather not  have a "match" than have it approximate. the
# goal here is to be as close to the pose as possible!
with mp_holistic.Holistic(min_detection_confidence=0.95) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections, this will call the model with the actions.h5 data from the training
        # and send us an array saying what was the match ratio for each of the poses it knows about.
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks, so the user can see it on screen. This won't be helpful for a blind user
        # this is just here for the developers/us to make sure the code is working correctly and 
        # landmarks are being detected.
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # If we have captured 30 frames of data, use the last 30 frames for prediction
        # this is what makes our model detect the fluidity of motion... it will detect the way
        # we train it... 30 frames == 1 sec. So, our pose can be a 1 second video - which will be
        # matched with the trained data.
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # Did we have a match?
            if res[1] > threshold:
                print(res)
                pose_achieved.play()
                cv2.waitKey(10000)
                break
            # Did we have a close match? If yes, beep faster.
            elif res[1] > threshold1 :
                print(res)
                BeepTwoPerSec.play()
                cv2.waitKey(1000)
                BeepTwoPerSec.stop()
            # Did we have a rough match? Beep slower, so the user knows they are getting close.
            elif res[1] > threshold2:
                print(res)
                BeepOnePerSec.play()
                cv2.waitKey(1000)
                BeepOnePerSec.stop()

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
[3.0742271e-12 9.9214727e-01 7.8487629e-03 4.0268578e-06]
1/1 [==============================] - 0s 23ms/step
[2.1110845e-13 9.9931049e-01 6.8927463e-04 2.6454782e-07]
1/1 [==============================] - 0s 23ms/step
[1.41651827e-14 9.99963403e-01 3.65888773e-05 1.00110435e-08]
